In [2]:
import librosa
import matplotlib.pyplot as plt
import IPython.display as dis
import numpy as np
import os, sys
from tqdm import tqdm

ModuleNotFoundError: No module named 'librosa'

In [90]:
# 경로 설정
path = 'C:\\Users\\인공지능사관학교\\Desktop\\최종프로젝트\\gtzan\\Data\\genres_original'

wholedata = []
genres = []

root_dir = os.listdir(path)
for categori in tqdm(root_dir):
    categori_path = path+'/'+str(categori)
    genres.append(categori)
    categori_dir = os.listdir(categori_path)
    # 스펙트로그램을 담을 임시 리스트
    sub_list = []
    for music in categori_dir:
        # wav 파일을 가져와 멜스펙토그램으로 변환한후 리스트에 담는다.
        y, sr = librosa.load(categori_path+'/'+str(music))
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        S_DB = librosa.amplitude_to_db(S, ref=np.max)
        sub_list.append(S_DB)
    
    wholedata.append(sub_list)



100%|██████████| 10/10 [00:42<00:00,  4.24s/it]


In [97]:
wav_to_img_data = []
target = []
cnt = 0
# 128*128 사이즈로 멜스펙토그램 이미지를 잘라서 담아두고
# 타겟값을 같이 추가한 넘파이배열을 만든다
for categori in tqdm(wholedata):
    for spec in categori:
        for i in range(5):
            wav_to_img_data.append(spec[:, 128* i : 128 * (i+1)].reshape(128,128,-1))
            target.append(cnt)
    cnt += 1
wav_to_img_data = np.array(wav_to_img_data)
target = np.array(target)
del cnt

100%|██████████| 10/10 [00:00<00:00, 1671.10it/s]


In [74]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten

In [139]:
# layers

inputs = Input(shape=(128,128,1))

conv_1 = Conv2D(16, kernel_size=3, strides=1, padding='same')(inputs)
bn_1 = BatchNormalization()(conv_1)
relu_1 = Activation('relu')(bn_1)
maxpool_1 = MaxPooling2D(pool_size=2, padding='same')(relu_1)

conv_2 = Conv2D(32, kernel_size=3, strides=1, padding='same')(maxpool_1)
bn_2 = BatchNormalization()(conv_2)
relu_2 = Activation('relu')(bn_2)
maxpool_2 = MaxPooling2D(pool_size=2, padding='same')(relu_2)

conv_3 = Conv2D(64, kernel_size=3, strides=1, padding='same')(maxpool_2)
bn_3 = BatchNormalization()(conv_3)
relu_3 = Activation('relu')(bn_3)
maxpool_3 = MaxPooling2D(pool_size=2, padding='same')(relu_3)

flatten = Flatten()(maxpool_3)
dense_1 = Dense(128, activation='relu')(flatten)
dense_2 = Dense(10, activation='softmax')(dense_1)

In [140]:
model = tf.keras.models.Model(inputs, dense_2)

In [141]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [142]:
history = model.fit(wav_to_img_data, target, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
125/125 [==============================] - 53s 425ms/step - loss: 3.1974 - acc: 0.3371 - val_loss: 8.4706 - val_acc: 0.0000e+00
Epoch 2/10
125/125 [==============================] - 56s 445ms/step - loss: 1.1697 - acc: 0.5906 - val_loss: 11.3976 - val_acc: 0.0000e+00
Epoch 3/10
125/125 [==============================] - 55s 437ms/step - loss: 0.9115 - acc: 0.6789 - val_loss: 12.7753 - val_acc: 0.0000e+00
Epoch 4/10
125/125 [==============================] - 54s 434ms/step - loss: 0.7095 - acc: 0.7548 - val_loss: 13.7224 - val_acc: 0.0000e+00
Epoch 5/10
125/125 [==============================] - 55s 436ms/step - loss: 0.5777 - acc: 0.8083 - val_loss: 11.7431 - val_acc: 0.0000e+00
Epoch 6/10
125/125 [==============================] - 55s 442ms/step - loss: 0.4381 - acc: 0.8531 - val_loss: 12.5806 - val_acc: 0.0000e+00
Epoch 7/10
125/125 [==============================] - 57s 452ms/step - loss: 0.3797 - acc: 0.8696 - val_loss: 11.2596 - val_acc: 0.0000e+00
Epoch 8/10
125/125 [=

In [87]:
wav_to_img_data.shape

(0,)

In [149]:
wholecnt = 0
correctcnt = 0
for i in range(10):
    for j in range(10):
        k = np.random.randint(0,80)
        l = np.random.randint(0,1000)
        sample = wholedata[i][k][:,l:l+128]
        pred = np.argmax(model.predict(sample.reshape(1,128,128,1)))
        if pred == i:
            correctcnt +=1
        wholecnt+=1



In [150]:
correctcnt/wholecnt

0.52